#Advanced RAG with Merger Retriever (LOTR) and Re-Ranking retriever (For long context reorder)

In [1]:
!pip install -q langchain langchain_core chromadb huggingface_hub transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies

In [3]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.1 MB/s eta 0:00:00


In [6]:
!pip install -q sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 49.9 MB/s eta 0:00:00


In [9]:
!pip install -q pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 2.1 MB/s eta 0:00:00


In [4]:
import os
import chromadb
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_transformers import (
    EmbeddingsRedundantFilter,
    EmbeddingsClusteringFilter,
)
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Get the Embedding Model

In [7]:
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
print("Embedding Model Loaded..........")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Embedding Model Loaded..........


## Data Preprocessing

In [12]:
loader_un_sdg = PyPDFLoader("/content/UN SDG.pdf")
documents_un_sdg = loader_un_sdg.load()
text_splitter_un_sdg = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                   chunk_overlap=100)
texts_un_sdg = text_splitter_un_sdg.split_documents(documents_un_sdg)

In [13]:
texts_un_sdg[0]

Document(metadata={'source': '/content/UN SDG.pdf', 'page': 0}, page_content='4th SDG Youth Summer Camp – SDG Resource Document  \nThe 2030 Agenda for Sustainable Development’s 17 \nSustainable Development Goals (SDGs)    \nGoal: This document enables 4th SDG Youth Summer Camp participants to i) get to \nknow the 17 SDGs, ii) explore what areas each goal covers under its targets, iii) identify \ntargets of most interest to participants, and iv) identify synergies between the SDGs \nand chosen target(s).  \n \n \nGoal 1. End poverty in all its forms everywhere   \nTarget 1.1  By 2030, eradicate extreme poverty  for all people everywhere, currently measured \nas people living on less than $1.25 a day  \nTarget 1.2  By 2030, reduce at least by half the proportion  of men, women and  children of all \nages living in poverty  in all its dimensions according to national definitions  \nTarget 1.3  Implement nationally appropriate social protection systems and measures for \nall, including flo

In [15]:
loader_paris_agreement = PyPDFLoader("/content/english_paris_agreement.pdf")
documents_paris_agreement = loader_paris_agreement.load()
text_splitter_paris_agreement = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                   chunk_overlap=100)
texts_paris_agreement = text_splitter_paris_agreement.split_documents(documents_paris_agreement)

In [16]:
texts_paris_agreement[0]

Document(metadata={'source': '/content/english_paris_agreement.pdf', 'page': 0}, page_content='PARIS AGREEMENT \n(mm \nUNITED NATIONS \n2015')

## Create and Store Vectors

In [17]:
un_sdg_store = Chroma.from_documents(texts_un_sdg, hf, collection_metadata={"hnsw:space": "cosine"}, persist_directory="/content/un_sdg_chroma_cosine")

In [19]:
paris_agreement_store = Chroma.from_documents(texts_paris_agreement, hf, collection_metadata={"hnsw:space": "cosine"}, persist_directory="/content/paris_chroma_cosine")

## Load Vector Store

In [20]:
load_un_sdg_store = Chroma(persist_directory="/content/un_sdg_chroma_cosine", embedding_function=hf)

In [21]:
load_paris_agreement_store = Chroma(persist_directory="/content/paris_chroma_cosine", embedding_function=hf)

## Init Merge Retriever

In [26]:
retriever_un_sdg = load_un_sdg_store.as_retriever(search_type = "similarity", search_kwargs = {"k":3})

retriever_paris_agreement = load_paris_agreement_store.as_retriever(search_type = "similarity", search_kwargs = {"k":3})

In [27]:
lotr = MergerRetriever(retrievers=[retriever_un_sdg, retriever_paris_agreement])

In [28]:
lotr

MergerRetriever(retrievers=[VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x792499830ac0>, search_kwargs={'k': 3}), VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x792499831390>, search_kwargs={'k': 3})])

## Perform Semantic Search

In [29]:
for chunks in lotr.get_relevant_documents("Is there any framework available to tackle the climate change?"):
    print(chunks.page_content)

Goal 13. Take urgent action to combat climate change and its impacts 
Target 13.1 Strengthen resilience and adaptive capacity to climate-related hazards and 
natural disasters  in all countries  
Target 13.2 Integrate climate change measures into national policies,  strategies and 
planning  
Target 13.3 Improve education, awareness-raising and human and institutional capacity 
on climate change mitigation, adaptation, impact reduction and early warning  
Target 13.a Implement the commitment undertaken by developed-country parties  to the 
United Nations Framework Convention on Climate Change to a goal of mobilizing jointly $100 
billion annually by 2020 from all sources to address the needs of developing countries in the 
context of meaningful mitigation actions and transparency on implementation and fully 
operationalize the Green Climate Fund through its capitalization as soon as possible  
Target 13.b Promote mechanisms  for raising capacity for effective climate change-related
fin

In [30]:
for chunks in lotr.get_relevant_documents("What is Paris Agreement?"):
    print(chunks.page_content)

environmentally sound technologies  to developing countries on favourable terms, including 
on concessional and preferential terms, as mutually agreed  
Target 17.8 Fully operationalize the  technology bank and science, technology and innovation 
capacity-building mechanism for least developed countries  by 2017 and enhance the use 
of enabling technology, in particular information and communications technology 
Capacity-building   
Target 17.9 Enhance international support for implementing effective and targeted  
capacity-building in developing countries to support national plans  to implement all the 
Sustainable Development Goals , including through North-South, South-South and triangular 
cooperation
PARIS AGREEMENT 
(mm 
UNITED NATIONS 
2015
Target 13.b Promote mechanisms  for raising capacity for effective climate change-related 
planning and management  in least developed countries and small island developing States, 
including focusing on women, youth and local and marginalize